# INST447: Data Sources and Manipulation
## Lecture 09: LLM Tool Use - When LLMs Need to Access the Real World

Today's Topics:
- What is tool use and why does it matter?
- How tool use works (the orchestration pattern)
- Hands-on: Building a calculator for precise arithmetic
- Hands-on: Building a weather information bot with real API
- (Optional) Nutritional information lookup with OpenFoodFacts
- (Optional) Multi-tool usage: combining capabilities
- What makes tool use possible: reasoning, context, and learning

In [ ]:
# Setup imports
import json
import os
import requests
from openai import OpenAI

# API key setup
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'your-api-key-here')
client = OpenAI(api_key=OPENAI_API_KEY)

## Part 1: Motivation - The Limits of Pure Language Models

### Recap: What We've Learned

**Lecture 06: JSON**
- Structured data format for nested/hierarchical information
- Parsing and working with JSON in Python

**Lecture 07: APIs**
- How to fetch data from web APIs (OpenFoodFacts example)
- Making HTTP requests with the requests library
- Parsing JSON responses

**Lecture 08: LLM APIs**
- How to call LLM APIs (OpenAI, OpenRouter)
- Using LLMs to augment data (sentiment analysis, classification)
- Applying LLMs to pandas DataFrames

**Today: Combining LLMs + Regular APIs = Intelligent Agents**


### The Problem with LLMs Alone

LLMs have fundamental limitations:

1. **Bad at precise computation**
2. **No real-time information**
3. **No access to your private data**
4. **Can't take actions**

Let's demonstrate these limitations...

### Demo: LLM WITHOUT Tools - Demonstrating the Problem

In [ ]:
# Test 1: Large number calculation
# Question 1: What is 82374923 multiplied by 93478234?

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is 82374923 multiplied by 93478234? Just give me the exact numerical answer."}
    ]
)

In [ ]:
response.choices[0].message.content

In [ ]:
correct_answer = 82374923 * 93478234
correct_answer

In [ ]:
# Test 2: Real-time information
# Question 2: What's the weather like in Seattle right now?

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. DO NOT use any tools."},
        {"role": "user", "content": "What's the weather like in College Park MD right now? Give me the current temperature."}
    ]
)

In [ ]:

llm_answer = response.choices[0].message.content
print(llm_answer)


In [ ]:
# Get actual weather for comparison
geo_response = requests.get(
    "https://geocoding-api.open-meteo.com/v1/search",
    params={"name": "College Park", "count": 1, "format": "json"}
)
geo_data = geo_response.json()
lat, lon = geo_data["results"][0]["latitude"], geo_data["results"][0]["longitude"]

weather_response = requests.get(
    "https://api.open-meteo.com/v1/forecast",
    params={"latitude": lat, "longitude": lon, "current": "temperature_2m", "temperature_unit": "fahrenheit"}
)
actual_temp = weather_response.json()["current"]["temperature_2m"]

In [ ]:
geo_response.json()

In [ ]:
weather_response.json()


The LLM is a powerful *brain*, but it's trapped in a "jar." It has no eyes to see the current world, no hands to interact with it.

**Tool Use** gives LLM hands and eyes. It's the bridge that connects the LLM's reasoning to the real world.

### The Solution: Tool Use

At its simplest, **tool use** (or "function calling") is the ability for an LLM to "pause" its text generation, request a specific *function* be run, and then resume its generation once it gets the *result* of that function.

Instead of just *writing* code, it can now *request the execution* of code.

Give the LLM access to FUNCTIONS it can call when it needs:
- **Computation abilities**: Calculator for precise arithmetic
- **Real-time data**: Weather API, stock API, news API, etc.
- **Your private data**: Database queries, file reading, etc.
- **Action capabilities**: Send email, book appointment, etc.

This is the fundamental building block for creating **LLM Agents**. An Agent is a system that uses an LLM as its reasoning "brain" to plan and execute tasks, using tools to get things done.

## Part 2: What is Tool Use? Understanding the Pattern

### The Tool Use Pattern

**Without Tools:**
```
User: "What's 82374923 times 93478234?"
  ↓
LLM: "Approximately 7.7 × 10^15" (might be wrong!)
```

**With Tools:**
```
User: "What's 82374923 times 93478234?"
  ↓
LLM: "I need to call the calculator function with 82374923 * 93478234"
  ↓
Your code: Executes calculator → 7701234897623082 (exact!)
  ↓
Your code: Sends results back to LLM
  ↓
LLM: "The exact answer is 7,701,234,897,623,082"
```

### The "ReAct" Loop -- Reasoning and Acting

Thought $\rightarrow$ Action $\rightarrow$ Observation $\rightarrow$ Thought...

- **Thought**: The LLM's internal monologue (a special prompt). "The user wants the weather in College Park. I need to find a tool. I have a tool called `get_current_weather`. I should call it with the argument `city='College Park'`."

- **Action**: The LLM's *output*. It doesn't output text. It outputs a structured call, like: `{"tool_name": "get_current_weather", "arguments": {"city": "College Park"}}`.

- **Observation**: *Your code* catches this "action." You parse the JSON, run your *actual* Python `get_current_weather("College Park")` function, which calls an API and gets a result: `{"temperature": "60°F", "conditions": "Cloudy"}`. This result is the "observation."

- **Thought (Round 2)**: This observation is fed back to the LLM. The LLM thinks: "Okay, I have the observation. The weather is 60°F and cloudy. The user just wanted to know the weather. I can now form the final answer."

- ... More action/observatin/thought if needed...

- **Final Response**: The LLM generates the natural language response: "The current weather in London is 15°C and cloudy."

**How Does the LLM "Know" About Your Tools? (10 mins):**

- You don't give the LLM your Python code. You give it a "menu" or "schema" of the tools it's allowed to use.

- This is just a special part of the **system prompt**. You're "prompt engineering" the LLM to be a tool-using agent.

- You provide a list of tool definitions, usually in a format like JSON Schema.

### Key Concepts

1. **Tool Definition**: You tell the LLM what functions are available
   - Function name: "calculate"
   - Description: "Perform arithmetic operations"
   - Parameters: number1, number2, operation
   - Return type: result

2. **Function Calling**: The LLM decides:
   - Should I use a tool? (vs. answering from knowledge)
   - Which tool should I use? (if multiple available)
   - What parameters should I pass?

3. **Tool Execution**: Your code actually runs the function
   - The LLM doesn't run anything - it just tells you what to run
   - You're in control of what actually gets executed (important for security!)
   - You perform the actual calculation, API call, file read, etc.

4. **Result Integration**:
   - You send the function result back to the LLM
   - LLM incorporates this into its response
   - May call more functions if needed (multi-step reasoning)

Tool definitions, function calls, and results are all passed as JSON structures.

Tool use is a Game-Changer to the LLM ecosystem:

- **Overcomes Knowledge Cutoff**: The LLM can query real-time APIs (weather, stocks, news).
- **Accesses Private Data**: The LLM can query your company's database, your Pandas DataFrame, or your personal files (with tools you provide).
- **Performs Actions**: The LLM can do things—send an email, save a file, post a tweet, book a flight.
- **Improves Accuracy**: LLMs are bad at precise math. Instead of asking it "What is $123.45 * 67.89$?", you can give it a calculator tool. It will reason "I should use the calculator" and call the tool, getting a perfect answer.

## Part 3: Example 1 - Calculator for Precise Arithmetic

We just saw that LLMs make arithmetic errors with large numbers.
Let's fix this by giving the LLM a calculator tool!

**The Solution:**
Give it access to Python's arithmetic operations!

### Step 1: Define the calculator function

In [ ]:
def calculate(num1: float, num2: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.

    Args:
        num1: First number
        num2: Second number
        operation: One of "add", "subtract", "multiply", "divide"

    Returns:
        Dictionary with result or error
    """
    try:
        if operation == "add":
            result = num1 + num2
        elif operation == "subtract":
            result = num1 - num2
        elif operation == "multiply":
            result = num1 * num2
        elif operation == "divide":
            if num2 == 0:
                return {"error": "Division by zero"}
            result = num1 / num2
        else:
            return {"error": f"Unknown operation: {operation}"}

        return {
            "num1": num1,
            "num2": num2,
            "operation": operation,
            "result": result
        }
    except Exception as e:
        return {"error": str(e)}


# Test the calculator independently first!
print(calculate(82374923, 93478234, "multiply"))
print(calculate(123, 456, "add"))
print(calculate(100, 3, "divide"))

### Step 2: Create tool specification for OpenAI

This is the JSON structure that tells the LLM about our function.
Notice how this is similar to API documentation!

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform basic arithmetic operations (add, subtract, multiply, divide) on two numbers. Use this for precise calculations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "num2": {
                        "type": "number",
                        "description": "The second number"
                    },
                    "operation": {
                        "type": "string",
                        "description": "The operation to perform",
                        "enum": ["add", "subtract", "multiply", "divide"]
                    }
                },
                "required": ["num1", "num2", "operation"]
            }
        }
    }
]

print("=" * 80)
print("Tool specification (this is what we send to the LLM)")
print("=" * 80)
print(json.dumps(tools, indent=2))
print()

### Step 3: Make initial API call with tools available

Let's see what happens when we ask for a calculation and give the LLM access to our tool.
We'll break this down into small steps to see exactly what's happening.

In [ ]:
user_question = "What is 82374923 multiplied by 93478234?"

# Build the messages
messages = [
    {
        "role": "system",
        "content": "You are a helpful math assistant. When users ask for calculations, use the calculate function to get precise results."
    },
    {
        "role": "user",
        "content": user_question
    }
]

# Make the API call with tools available
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Let the model decide whether to use tools
)

In [ ]:

response.choices

### Step 4: Check the response - did LLM want to call a function?

In [ ]:
response_message = response.choices[0].message

print("=" * 80)
print("STEP 4: Examining the LLM's response")
print("=" * 80)
print(f"Response message role: {response_message.role}")
print(f"Response message content: {response_message.content}")
print(f"Has tool calls? {response_message.tool_calls is not None}")
print()

if response_message.tool_calls:
    print("Tool calls requested by LLM:")
    for tool_call in response_message.tool_calls:
        print(f"  Tool call ID: {tool_call.id}")
        print(f"  Function name: {tool_call.function.name}")
        print(f"  Function arguments (JSON string): {tool_call.function.arguments}")
        print(f"  Parsed arguments: {json.loads(tool_call.function.arguments)}")
    print()

### Step 5: Execute the function call

In [ ]:
print("=" * 80)
print("STEP 5: Executing the function")
print("=" * 80)

# The LLM wants to call a function - let's execute it
tool_call = response_message.tool_calls[0]
function_name = tool_call.function.name
function_args = json.loads(tool_call.function.arguments)

print(f"Function to call: {function_name}")
print(f"Arguments: {function_args}")
print()

# Actually execute the function
if function_name == "calculate":
    function_result = calculate(**function_args)
else:
    function_result = {"error": "Unknown function"}

print(f"Function result:")
print(json.dumps(function_result, indent=2))
print()

### Step 6: Add function result to conversation

In [ ]:
print("=" * 80)
print("STEP 6: Adding function result to conversation")
print("=" * 80)

# First, add the assistant's message (which contains the tool call)
messages.append(response_message)

# Then add the function result
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "name": function_name,
    "content": json.dumps(function_result)
})

print("Updated conversation messages:")
for i, msg in enumerate(messages):
    print(f"\nMessage {i}:")
    if isinstance(msg, dict):
        print(f"  Role: {msg['role']}")
        if 'content' in msg and msg['content']:
            content = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
            print(f"  Content: {content}")
    else:
        print(f"  Role: {msg.role}")
        if msg.content:
            content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
            print(f"  Content: {content}")
        if msg.tool_calls:
            print(f"  Tool calls: {len(msg.tool_calls)} function(s)")
print()

### Step 7: Get final response from LLM

In [ ]:
print("=" * 80)
print("STEP 7: Getting final response from LLM")
print("=" * 80)

# Make another API call with the function result included
final_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

final_answer = final_response.choices[0].message.content

print(f"Final response to user:")
print(final_answer)
print()
print("=" * 80)
print()

In [ ]:
82374923*93478234

### Wrapping into a reusable function

Now let's wrap this into a reusable function for convenience.

In [ ]:
def chat_with_calculator(user_message: str) -> str:
    """
    Chat with the calculator bot (simplified version).

    Args:
        user_message: User's question

    Returns:
        Bot's response
    """
    messages = [
        {"role": "system", "content": "You are a helpful math assistant. Use calculate for precise arithmetic."},
        {"role": "user", "content": user_message}
    ]

    # Step 1: Initial call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message

    # If no tool calls, return the text response
    if not response_message.tool_calls:
        return response_message.content

    # Step 2: Execute tool calls
    messages.append(response_message)

    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "calculate":
            function_result = calculate(**function_args)
        else:
            function_result = {"error": "Unknown function"}

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(function_result)
        })

    # Step 3: Get final response
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return final_response.choices[0].message.content

### Demo: Calculator bot in action

In [ ]:
print("=" * 80)
print("DEMO: Calculator bot in action")
print("=" * 80)
print()

# Example 1: Large multiplication
print("User: What is 82374923 multiplied by 93478234?")
response = chat_with_calculator("What is 82374923 multiplied by 93478234?")
print(f"Bot: {response}")
print()

print("-" * 80)
print()

# Example 2: Different operation
print("User: Calculate 12345 plus 67890")
response = chat_with_calculator("Calculate 12345 plus 67890")
print(f"Bot: {response}")
print()

print("-" * 80)
print()

# Example 3: Division
print("User: What's 100 divided by 7?")
response = chat_with_calculator("What's 100 divided by 7?")
print(f"Bot: {response}")
print()

print("=" * 80)
print()

## Part 4: Example 2 - Weather Bot with Real API

Now let's build something that needs external data: a chatbot that can tell you
REAL weather information by calling the Open-Meteo API.

**Open-Meteo API**: https://open-meteo.com/
- Free, no API key needed
- Provides current weather, forecasts, historical data
- We'll use their geocoding API to get coordinates, then weather data

### Define the weather function (REAL API call)

In [ ]:
def get_weather(location: str) -> dict:
    """
    Get current weather for a location using Open-Meteo API.

    This is a REAL function that makes REAL API calls!

    Args:
        location: City name (e.g., "College Park", "New York")

    Returns:
        Dictionary with temperature, weather description, wind speed, humidity
    """
    # Step 1: Get coordinates for the location using geocoding API
    geocoding_url = "https://geocoding-api.open-meteo.com/v1/search"
    geocoding_params = {
        "name": location,
        "count": 1,
        "language": "en",
        "format": "json"
    }

    geo_response = requests.get(geocoding_url, params=geocoding_params)
    geo_data = geo_response.json()

    if "results" not in geo_data or len(geo_data["results"]) == 0:
        return {"error": f"Location '{location}' not found"}

    # Extract coordinates
    lat = geo_data["results"][0]["latitude"]
    lon = geo_data["results"][0]["longitude"]
    location_name = geo_data["results"][0]["name"]

    # Step 2: Get weather data for these coordinates
    weather_url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
        "temperature_unit": "fahrenheit",
        "wind_speed_unit": "mph"
    }

    weather_response = requests.get(weather_url, params=weather_params)
    weather_data = weather_response.json()

    # Step 3: Parse and format the results
    current = weather_data["current"]

    # Weather code mapping (simplified)
    weather_codes = {
        0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
        45: "Foggy", 48: "Foggy", 51: "Light drizzle", 53: "Moderate drizzle",
        55: "Dense drizzle", 61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
        71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow", 80: "Rain showers",
        81: "Rain showers", 82: "Heavy rain showers", 95: "Thunderstorm"
    }

    weather_code = current.get("weather_code", 0)
    weather_description = weather_codes.get(weather_code, "Unknown")

    return {
        "location": location_name,
        "temperature_f": current["temperature_2m"],
        "weather": weather_description,
        "humidity": current["relative_humidity_2m"],
        "wind_speed_mph": current["wind_speed_10m"]
    }


# Test our function independently first!
print("=" * 80)
print("Testing the weather function directly (without LLM)")
print("=" * 80)

weather = get_weather("College Park")
print(f"Weather in {weather['location']}:")
print(f"  Temperature: {weather['temperature_f']}°F")
print(f"  Conditions: {weather['weather']}")
print(f"  Humidity: {weather['humidity']}%")
print(f"  Wind Speed: {weather['wind_speed_mph']} mph")
print()

### Define weather tool and create bot function

In [ ]:
# Define weather tool
weather_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a specific location. Use this when the user asks about weather conditions, temperature, or climate in a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g., 'College Park', 'New York', 'San Francisco'"
                    }
                },
                "required": ["location"]
            }
        }
    }
]


def chat_with_weather_bot(user_message: str) -> str:
    """Chat with weather bot (same pattern as calculator)."""
    messages = [
        {"role": "system", "content": "You are a helpful weather assistant. Use get_weather for current weather."},
        {"role": "user", "content": user_message}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=weather_tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message

    if not response_message.tool_calls:
        return response_message.content

    messages.append(response_message)

    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "get_weather":
            function_result = get_weather(**function_args)
        else:
            function_result = {"error": "Unknown function"}

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(function_result)
        })

    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return final_response.choices[0].message.content

### Demo: Weather bot

In [ ]:
print("=" * 80)
print("DEMO: Weather bot")
print("=" * 80)
print()

print("User: What's the weather like in Seattle?")
response = chat_with_weather_bot("What's the weather like in Seattle?")
print(f"Bot: {response}")
print()

print("-" * 80)
print()

# This should trigger TWO function calls (Seattle and Miami)
print("User: Should I visit Seattle or Miami based on the weather?")
response = chat_with_weather_bot("Should I visit Seattle or Miami based on the weather?")
print(f"Bot: {response}")
print()

print("=" * 80)
print()

## Part 5 (OPTIONAL): Example 3 - OpenFoodFacts Integration

Remember OpenFoodFacts from Lecture 07? Let's give an LLM access to it!

This shows how tool use can integrate with any API you've learned about.

In [ ]:
def get_nutrition_info(product_name: str) -> dict:
    """
    Search for a product on OpenFoodFacts and return nutritional information.

    Args:
        product_name: Name of the food product

    Returns:
        Dictionary with nutritional information
    """
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "page_size": 1,
        "json": 1
    }

    response = requests.get(search_url, params=params)
    data = response.json()

    if data["count"] == 0:
        return {"error": f"Product '{product_name}' not found"}

    product = data["products"][0]
    nutriments = product.get("nutriments", {})

    return {
        "product_name": product.get("product_name", "Unknown"),
        "brands": product.get("brands", "Unknown"),
        "calories_per_100g": nutriments.get("energy-kcal_100g", "N/A"),
        "fat_per_100g": nutriments.get("fat_100g", "N/A"),
        "carbs_per_100g": nutriments.get("carbohydrates_100g", "N/A"),
        "protein_per_100g": nutriments.get("proteins_100g", "N/A"),
        "nutriscore": product.get("nutriscore_grade", "N/A").upper()
    }


# Test it
print("=" * 80)
print("(OPTIONAL) Testing OpenFoodFacts function")
print("=" * 80)
print(json.dumps(get_nutrition_info("Nutella"), indent=2))
print()

# The tool definition and bot function would follow the same pattern...

## Part 6: Multi-Tool Usage - Combining Everything

### Multi-Tool Usage: The Power of Orchestration

The real power of tool use: LLMs can orchestrate multiple tools!

**Conceptual Example:**
If you give an LLM access to both weather and calculator tools, and ask:
"If it's 85°F in Miami, what's that in Celsius?"

The LLM would:
1. Call get_weather("Miami") to get actual temperature
2. Call calculate with the formula (F - 32) * 5/9
3. Synthesize: "It's currently X°F in Miami, which equals Y°C"

**How it works:**
- Same pattern as before, but now tools list contains multiple functions
- LLM decides which tool(s) to use based on the user query
- May call multiple tools in sequence
- You still execute each function and send results back

### Multi-Tool Usage: Combining Tools

The real power of tool use: LLMs can use multiple tools together!

**Example 1: Compare temperatures in two cities**
- Query: "Compare temperatures in Seattle and Miami, which is warmer?"
- LLM would:
  1. Call get_weather('Seattle')
  2. Call get_weather('Miami')
  3. Compare results and answer

**Example 2: Temperature conversion**
- Query: "If Seattle is 58°F, convert that to Celsius"
- LLM would:
  1. Call get_weather('Seattle') to confirm temperature
  2. Call calculate(58, 32, 'subtract') → 26
  3. Call calculate(26, 5, 'multiply') → 130
  4. Call calculate(130, 9, 'divide') → 14.44°C

The pattern is the same, just with more tools available!

## Part 7: What Makes Tool Use Possible?

### The Technology Behind Tool Use

How do LLMs "learn" to use tools? They weren't explicitly trained on function calling!

### Three Key Enabling Factors

**1. Reasoning Capacity**
- Modern LLMs have strong reasoning abilities
- Can understand: "I need external information to answer this"
- Can plan: "First get weather for A, then B, then compare"
- Can interpret function descriptions and match them to user needs

**2. Long Context Windows**
- Early models: ~4K tokens context
- Modern models: 128K - 1M+ tokens
- Allows LLMs to see:
  - Tool definitions
  - Conversation history
  - Function results
  - User query
  All at once!

**3. Instruction Following**
- LLMs trained to follow complex instructions
- Can understand: "When you need information, output JSON with function name and args"
- Can adapt to different tool formats and conventions

### How LLMs Learn Tool Use

**Through Alignment Training:**

1. **Fine-tuning on tool use examples**
   - Trained on millions of examples of function calling
   - Learns pattern: Question → Recognize need → Call function → Use result

2. **Reinforcement Learning from Human Feedback (RLHF)**
   - Reward correct tool usage
   - Penalize making up information when a tool is available
   - Penalize calling wrong function or wrong parameters

3. **Synthetic data generation**
   - Generate diverse tool-use scenarios
   - Cover edge cases and error handling

**Important**: The LLM doesn't actually "run" code or "make" API calls.
It outputs JSON that YOUR code interprets and executes. This is key for security!

### Real-World Agent Examples

**1. ChatGPT**
- Web search, code interpreter, image generation, DALL-E
- LLM orchestrates which plugin to use when

**2. Claude Code**
- File operations, terminal commands, code editing
- Decides when to read files, when to edit, when to run commands

**3. Corporate AI assistants**
- Access to internal knowledge bases (documents, wikis)
- Integration with company systems (HR, IT, CRM)
- Example: "What's the status of my PTO request?"
  → Queries HR system → Returns answer

**4. Customer service bots**
- Order tracking, inventory lookup, refund processing
- Orchestrate multiple backend systems

**Key insight**: Tool use + reasoning capacity + long context = capable AI agents

## Part 8: Best Practices & Wrap-up

### Key Considerations

**1. Security**
- You control what executes - LLM only suggests
- Always validate inputs before execution
- Restrict tools to safe operations only

**2. Cost & Reliability**
- Each tool call = multiple API requests
- LLMs may occasionally call wrong function
- Test functions independently first

**3. Error Handling**
- Always handle errors gracefully
- Return clear error messages to LLM
- LLM can often recover from errors

### Summary: The Tool Use Pattern

1. **Define tools**: Create function + JSON specification
2. **LLM decides**: Model determines if/when/how to use tools
3. **You execute**: Your code runs the actual function
4. **LLM synthesizes**: Model incorporates results into response

This pattern transforms LLMs from text generators into capable agents!

## Key takeaways:
1. Tool use gives LLMs access to external capabilities
2. The pattern: Define → LLM decides → You execute → LLM synthesizes
3. JSON is central to tool definitions and results
4. Reasoning + Long context + Instruction following = Tool use
5. Tool use is what makes LLMs into agents

### Next Steps

- Experiment with these examples
- Spot tool calling in using ChatGPT
- Think about what tasks could benefit from tool use
- Explore OpenAI's function calling documentation